In [24]:
import os
import random
import collections
spam = []

def initLists(folder):
    a_list = []
    file_list = os.listdir(folder)
    for a_file in file_list:
        f = open(folder + a_file, 'r')
        a_list.append(f.read())
    f.close()
    return a_list

# Takes as input a list of emails and associated spam/ham labels
# and returns a tuple of label and a list of the words in a given email
def processEmails(emails):
    # Split the emails up into lists of words     
    emailVectors = [(email[0], email[1].split()) for email in allEmails]
    
    # Create a list to hold tuples containing label and associated list of words for each email      
    processedEmails = []
    
    for vector in emailVectors:
        wordsIn = vector[1]
        label   = vector[0]
        wordsOut = []
        for word in wordsIn:
            if(word.isalpha() == 1):
                wordsOut.append(word)          
        processedEmails.append((label, wordsOut))        
    
    return processedEmails;
    

# Takes as input list of labels and associated words
# and returns list of labels along with wordcount for each word in the email
def countWordOccurances(processedList):
    vectorList = []
    for email in processedList:
        label = email[0]
        words = email[1]
        
        counter = collections.Counter(words)
        vectorList.append((label,counter.items()))
    return vectorList

def probabilities(label,processedList):
    spamWords = []
    for email in processedList:
        if(label == email[0]):
            spamWords.append(email[1])
        l = [item for sublist in spamWords for item in sublist]    
    counter = collections.Counter(l).most_common(100) 
    return counter;

# read spam and ham from respective directories
spam = initLists("spam/")
ham  = initLists("Ham/")

# compose list of labels and associated emails
allEmails = [('spam', email) for email in spam]
allEmails += [('ham', email) for email in ham]
    
random.shuffle(allEmails)
procEmails = processEmails(allEmails)
# out = countWordOccurances(procEmails) 

print (probabilities("spam",procEmails))
print "------------------------------"
print (probabilities("ham",procEmails))

# print out[1]


[('the', 1519), ('to', 1446), ('you', 1248), ('and', 1046), ('of', 853), ('a', 752), ('for', 734), ('your', 585), ('is', 540), ('with', 537), ('this', 535), ('in', 508), ('from', 443), ('by', 416), ('it', 361), ('will', 349), ('that', 341), ('i', 334), ('have', 297), ('or', 295), ('on', 289), ('be', 281), ('are', 277), ('can', 256), ('Aug', 234), ('our', 229), ('we', 226), ('as', 198), ('if', 191), ('mail', 189), ('do', 183), ('all', 181), ('not', 172), ('at', 166), ('id', 159), ('out', 154), ('money', 143), ('my', 137), ('how', 134), ('send', 132), ('get', 131), ('program', 129), ('people', 126), ('any', 125), ('more', 124), ('make', 124), ('order', 118), ('report', 118), ('one', 117), ('they', 114), ('I', 113), ('an', 108), ('no', 108), ('email', 107), ('address', 104), ('ESMTP', 103), ('time', 102), ('so', 95), ('was', 95), ('only', 93), ('just', 92), ('free', 92), ('business', 92), ('name', 90), ('list', 89), ('information', 88), ('please', 84), ('receive', 83), ('each', 83), ('up'